In [1]:
import json

In [2]:
import sys
sys.path.append('../library')

from helpers import *

In [3]:
###
### Concatenteaza o lista de string.
### param list_: lista de string.
### return string-ul rezultat dupa concatenare.
###
def get_key_from_lists(list_):
    key = ""
    for elem in list_:
        key += elem + ' '
    return key[0:len(key)-1]

###
### Calculeaza distanta euclidiana dintre un vector si o lista de vectori si returneaza o lista cu cei mai aporpiati nr.
### param X: dictionar {cheie:vector}. -> vectorul
### param X_dict: dictionar {chei:vectori} -> vectorii.
### param nr: numarul de vecini cei mai apropiati pe care ii va returna.
### return distante: dictionar {chei:vectori}.
### 
def calculeaza_distante_p(X, X_dict, nr):
    distante = {}
    for elem in X_dict:
        distante[elem] = distance.euclidean(X, X_dict[elem])        
    distante = sort_dict_value(distante)
    return distante[0:nr]

###
### Parcurge toate fisierele si creeaza un dictionar {venue:[list_of_fos]}.
### file_path: calea spre fisiere.
### param print_: printeaza log la 10 iteratii.
### return dict_: dictionarul.
###
def get_dict_foses(file_path, print_ = False):
    dict_ = {}
    j = 0
    for i in range(0,169):
        if (print_):
            if j % 10 == 0:
                print(j)
            j += 1
        file = file_path + str(i) + '.txt'
        r = open(file,'r')
        
        for linie in r:
            articol_crt = json.loads(linie)
            venue = articol_crt['venue']
            venue = get_key_from_lists(venue)
            fos = articol_crt['fos']
            try:
                fos_list = dict_[venue]
            except KeyError:
                fos_list = []
            
            for elem in fos:
                fos_list.append(elem['name'])
            
            dict_[venue] = remove_duplicates(fos_list)

        r.close()
    return dict_

###
### Parcurge toate elementele unui dictionar de tipul {venues:[encoding]}
### param dict_1: dictionarul.
### param vecini: numarul de vecini cautati.
### return dict_2: dictionar ce contine vecini nr de vecini [venue:[list_of_venues]] 
### 
def get_dict_similar_p(dict_1, vecini, del_=True):
    dict_2 = {}
    k = 0    
    for key in dict_1:
        
        new_X_dict = elimina_elem_dupa_cheie(dict_1, key)
        distante = calculeaza_distante_p(dict_1[key],new_X_dict, vecini) 
        list_ = []
        
        key = key[0:len(key)-1]
        
        for elem in distante:
            if del_ == True:
                list_.append(elem[0][0:len(elem[0])-1])
            else:
                list_.append(elem[0])
        dict_2[key] = list_
    print(len(dict_2))
    return dict_2


def get_dict_similar_abstr(dict_1, vecini, del_=True):
    dict_2 = {}
    k = 0    
    for key in dict_1:
        
        new_X_dict = elimina_elem_dupa_cheie(dict_1, key)
        distante = calculeaza_distante_p(dict_1[key],new_X_dict, vecini) 
        list_ = []
        
        for elem in distante:
            if del_ == True:
                list_.append(elem[0])
            else:
                list_.append(elem[0])
        dict_2[key] = list_
    print(len(dict_2))
    return dict_2
    

In [4]:
def compute_union(d1, d2):
    union = []
    union += d1
    union += d2
    union = remove_duplicates(union)
    return len(union)

def jacard(dict_venue_fos, dict_venue_vecini, p = False):
    j = 0
    i =0
    
    for venue_crt in dict_venue_vecini:
        list_fos_vecini = []
        for venue in dict_venue_vecini[venue_crt]:
            list_fos_vecini+=dict_venue_fos[venue]    
            
        intersection = elemente_comune_lista(dict_venue_fos[venue_crt], list_fos_vecini)
        union = compute_union(dict_venue_fos[venue_crt], list_fos_vecini)
        j += (intersection/union)
        
        if p == True:
            if i % 100 == 0:
                print(i)
        i += 1
    return j


def process(dict_venue_fos, dict_venue_venues):   
    procent_total = 0
    k = 0
    valids = 0
    for elem in dict_venue_fos:
        
        l2 = []
        l1 = dict_venue_fos[elem] # toate fos pt venue curent
        
        try:
            for fos in dict_venue_venues[elem]: # caut fos de la vecini
                l2 += dict_venue_fos[fos]
        except KeyError:
            continue
        l2 = remove_duplicates(l2)
        valids += 1
        
        # Gasesc fos din venue curent care se gasesc si la vecini
        l_comun = elemente_comune_lista(l1,l2)
        
        # jacard pentru A = fos care se gasesc si la vecini, B = fos pentru venue curent
        proc = len(l_comun)/len(l1)
        procent_total += proc        

    procent_total /= valids
    return procent_total


In [5]:
dict_venue_fos = get_dict_foses('../Date/Initiale/Complet/file', False)

## Word2vec gensim avg

In [204]:
r = open('../Date/word2vec-gensim/Venue/w2v_gensim_avg_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3583
0.48373119857968316
3583
0.60693685608885
3583
0.6737468739826491
3583
0.7189918664205573


## Word2vec gensim sum

In [205]:
r = open('../Date/word2vec-gensim/Venue/w2v_gensim_sum_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3583
0.5199247488010289
3583
0.6513314723511857
3583
0.7161088588622757
3583
0.757171085436828


## Word2vec tensorflow avg 

In [6]:
r = open('../Date/word2vec-tensorflow/Venue/w2v_tf_avg_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3583
0.5853413101079363
3583
0.7066274244459667
3583
0.7644816978815915
3583
0.8005226829076263


## Word2vec tensorflow sum

In [7]:
r = open('../Date/word2vec-tensorflow/Venue/w2v_tf_sum_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3583
0.6033530480964197
3583
0.7261841622156537
3583
0.7850338977920095
3583
0.8202889869238266


## Word2vec pretrained avg

In [211]:
r = open('../Date/word2vec-pretrained/Venue/w2v_avg_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3582
0.5651990566801232
3582
0.6924654579370793
3582
0.7564209239091935
3582
0.7964951932591221


## Word2vec pretrained sum

In [212]:
r = open('../Date/word2vec-pretrained/Venue/w2v_sum_50_c.txt', 'r')
X_dict = json.load(r)
r.close()

# # dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_p(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3582
0.5758077083980073
3582
0.7072896469398123
3582
0.7725850128207393
3582
0.8122450315321679


## Doc2vec

In [214]:
r = open('../Date/doc2vec/Venue/d2v_venue_complete.txt', 'r')
X_dict = json.load(r)
r.close()

# dict cu {venue:[similar_venues]}
rang = [5,10,15,20]
for elem in rang:
    venues = get_dict_similar_abstr(X_dict,elem)

    j = process(dict_venue_fos, venues)
    print(j)

3583
0.622302801958108
3583
0.7485565617753093
3583
0.8078531925253469
3583
0.8425366455864072


In [6]:
venue_neig = {'a':['ana','mere'], 'b':['mere','si','pere']}
venue_fos = {'a':['1','2'],'ana':['2','3'], 'are':['4','1'], 'mere':['5', '6'], 'b':['2','5','1'], 'si':['6'],'pere':['6','2','3']}

In [ ]:
a : [1 2] v [2 3 5 6] i: [1 2] u: [2] r: 1/2
b : [1 2 5 ] v [2 3 5 6] i: [1 2 5] u: [2 5] r: 2/3

In [22]:
def process_p_2(dict_venue_fos, dict_venue_venues):   
    procent_total = 0
    k = 0
    valids = 0
    for elem in dict_venue_fos:
        
        l2 = []
        l1 = dict_venue_fos[elem] # toate fos pt venue curent
        
        try:
            for fos in dict_venue_venues[elem]: # caut fos de la vecini
                l2 += dict_venue_fos[fos]
        except KeyError:
            continue
        l2 = remove_duplicates(l2)
        valids += 1
        
        # Gasesc fos din venue curent care se gasesc si la vecini
        l_comun = elemente_comune_lista(l1,l2)
        
        # jacard pentru A = fos care se gasesc si la vecini, B = fos pentru venue curent
        proc = len(l_comun)/len(l1)
        print(l_comun, l1)
        print(proc)
        procent_total += proc        
    print(procent_total, valids)
    procent_total /= valids
    return procent_total


In [23]:
process_p_2(venue_fos, venue_neig)

['2'] ['1', '2']
0.5
['5', '2'] ['2', '5', '1']
0.6666666666666666
1.1666666666666665 2


0.5833333333333333